#### 설계: 전선

In [10]:
import os
import warnings
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# 사용자정의 함수: key를 입력하면 해당 데이터를 데이터프레임으로 읽어옴
from aiddd.data_manager import read_data, write_data
from freeman.plt_setting import plt_settings

# matplotlib 설정: 한글표시, 차트 레이블 '-' 처리
plt_settings()

In [11]:
df_line = read_data('설계 전선')
df_line.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9853 entries, 0 to 9852
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   공사번호           9853 non-null   int64  
 1   공사변경순번         9853 non-null   int64  
 2   전산화번호          9853 non-null   object 
 3   GISID          9853 non-null   int64  
 4   공사구분코드         9853 non-null   object 
 5   전원측전산화번호       9853 non-null   object 
 6   전압구분코드         9853 non-null   object 
 7   설치순서           9853 non-null   int64  
 8   공사성격코드         9853 non-null   object 
 9   회선명            0 non-null      float64
 10  결선방식코드         9853 non-null   int64  
 11  상배열구분코드        0 non-null      float64
 12  지지물간거리         9853 non-null   int64  
 13  전선종류코드1        9851 non-null   object 
 14  전선규격코드1        9851 non-null   float64
 15  전선조수1          9853 non-null   int64  
 16  전선구분코드1        9660 non-null   object 
 17  전선조정코드1        0 non-null      float64
 18  전선공사특성코드

#### 데이터전처리

##### 결측치 1차 처리

In [12]:
# 결측치 확인
df_line.isna().sum().sort_values(ascending=False)[20:40]

전선규격코드3      9853
전선구분코드3      9853
중성선조정코드      9853
전선구분코드2      9829
전선공사특성코드2    9824
전선종류코드2      9824
전선규격코드2      9824
중성선구분코드      4370
중성선공사특성코드    2497
전선구분코드1       193
중성선규격코드        14
중성선종류코드        14
전선규격코드1         2
전선종류코드1         2
중성선공용구분코드       1
최종변경일시          0
공사형태명           0
공사형태코드          0
접수종류명           0
접수종류코드          0
dtype: int64

In [13]:
# 결측치 갯 수가 4000개 이상인 컬럼 리스트
_nan_counts = df_line.isna().sum()
cols_with_most_nan = _nan_counts[_nan_counts >= 4000].index.tolist()
cols_with_most_nan

['회선명',
 '상배열구분코드',
 '전선조정코드1',
 '전선공사특성코드2',
 '전선종류코드2',
 '전선규격코드2',
 '전선구분코드2',
 '전선조정코드2',
 '중성선구분코드',
 '중성선조정코드',
 '직선접속공사특성코드',
 '시공방법코드',
 '건물방호관코드',
 '직선접속중성선공사성격코드',
 '전선공사특성코드3',
 '전선종류코드3',
 '전선규격코드3',
 '전선구분코드3',
 '전선조정코드3',
 '사업소코드',
 '변전소코드',
 '회선코드',
 '기설설비상태코드',
 '종단접속공사코드',
 '조가형태공사코드',
 '조가형태코드',
 '파형관규격코드',
 'NDIS사업소코드']

In [14]:
# 결측치 4000개 이상 컬럼 삭제
df_line.drop(columns=cols_with_most_nan, axis=1, inplace=True)

##### 유일값 컬럼 처리

In [15]:
# 값이 하나인 컬럼 확인
cols_unique = df_line.columns[df_line.nunique() == 1]

print(f'값이 하나인 컬럼 갯 수: {len(cols_unique)}')

값이 하나인 컬럼 갯 수: 20


In [16]:
cols_unique

Index(['공사변경순번', '공사구분코드', '전압구분코드', '공사성격코드', '전선펴기여부', '직선접속수량', '슬리브접속수량',
       '상부가설여부', '건물방호관수량', '고객제공여부', '직선접속중성선수량', '전압선중성선사용여부', '중성선전압선사용여부',
       '전선펴기긴선여부', '기설전선여부', '기설설비유실여부', '종단접속수량', '디지털장력계여부', '케이블헤더수량',
       '사업소명'],
      dtype='object')

In [17]:
# 값이 하나인 컬럼 제거
df_line.drop(columns=cols_unique, axis=1, inplace=True)
# write_data('전처리 설계전선', df_line)

##### 전처리된 공사비를 읽어와서 설계:전선 값과 비교

In [18]:
df_cc_pp = read_data('전처리 공사비')

In [19]:
df_cc_pp['공사번호'].nunique(), df_cc_pp.shape
# 공사번호별로 하나의 로우로 구성됨을 알 수 있음(공사번호 유일값 갯 수 = 전체 로우 갯 수)

(5810, (5810, 5))

In [20]:
# 설계:전선에서 중복된 공사번호의 중복 제거
unique_no_line = df_line['공사번호'].unique()
len(unique_no_line)

# 전체공사비 5810개, 설계:전선의 공사비 4289개 
# => 공사비중에 설계:전선이 없는 공사비: 1521개

4289

In [21]:
# 공사비에서 설계:전선의 유일한 공사번호로우 추출
df_cc_pp_line = df_cc_pp[df_cc_pp['공사번호'].isin(unique_no_line)]
len(df_cc_pp_line)

# 최종적으로 공사비와 설계:전선의 공사번호가 일치한 로우는 3653개

3653

In [22]:
# 공사비의 공사번호가 있는 설계:전선 레코드 추출
df_line_cc = df_line[df_line['공사번호'].isin(df_cc_pp_line['공사번호'])]
df_line_cc.shape

(7743, 28)

In [23]:
# 양 데이터프레임에서 '공사번호'가 같은 로우의 '계약전력'은 모두 일치함
# 따라서 공사번호당 계약전력은 일치한다고 볼 수 있음
df_line_cc_not_match = df_line_cc[~df_line_cc['계약전력'].isin(df_cc_pp_line['계약전력'])]
df_line_cc_not_match.shape

(0, 28)

In [24]:
sr_line_counts = df_line['공사번호'].value_counts()
sr_line_counts.sum()

9853

In [25]:
# 공사비 데이터프레임에 전선수 컬럼 추가
df_cc_pp = pd.merge(
    df_cc_pp, sr_line_counts.rename('전선수'), 
    left_on='공사번호', right_on=sr_line_counts.index,
    how='left'
)

In [26]:
df_cc_pp['전선수'] = df_cc_pp['전선수'].fillna(0)
df_cc_pp['전선수'].sum()
# 공사비의 공사번호가 있는 설계:전선 레코드 추출값인 7743과 동일함

7743.0

In [28]:
write_data('전처리 공사비+설계', df_cc_pp)